In [1]:
import json
import os
import tensorflow as  tf

os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["10.0.1.121:8890", "10.0.1.104:8890", "10.0.1.46:8890"]
    },
    'task': {'type': 'worker', 'index': 0}
})

strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

import tensorflow as  tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
%load_ext tensorboard

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

print(x_train[0].shape)

def build_rnn():
    model = Sequential()
    
    model.add(LSTM(128, input_shape=(x_train.shape[1:]), activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(10, activation='softmax')) # activation must be softmax for categorical cross entropy loss
    
    opt = tf.keras.optimizers.Adam(lr=1e-3, decay=0)
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,
             metrics=['accuracy'])
  
    return model

with strategy.scope():
    net = build_rnn()

INFO:tensorflow:Enabled multi-worker collective ops with available devices: ['/job:worker/replica:0/task:0/device:CPU:0', '/job:worker/replica:0/task:0/device:XLA_CPU:0']
INFO:tensorflow:Using MirroredStrategy with devices ('/job:worker/task:0',)
INFO:tensorflow:MultiWorkerMirroredStrategy with cluster_spec = {'worker': ['10.0.1.121:8890', '10.0.1.104:8890', '10.0.1.46:8890']}, task_type = 'worker', task_id = 0, num_workers = 3, local_devices = ('/job:worker/task:0',), communication = CollectiveCommunication.AUTO
(28, 28)


In [2]:
with tf.profiler.experimental.Profile('logs/fit/my_rnn'):
    net.fit(x_train, y_train, batch_size = 100,
        epochs=3, validation_data=(x_test, y_test))
    pass

INFO:tensorflow:Running Distribute Coordinator with mode = 'independent_worker', cluster_spec = {'worker': ['10.0.1.121:8890', '10.0.1.104:8890', '10.0.1.46:8890']}, task_type = 'worker', task_id = 0, environment = None, rpc_layer = 'grpc'
INFO:tensorflow:Using MirroredStrategy with devices ('/job:worker/task:0',)
INFO:tensorflow:MultiWorkerMirroredStrategy with cluster_spec = {'worker': ['10.0.1.121:8890', '10.0.1.104:8890', '10.0.1.46:8890']}, task_type = 'worker', task_id = 0, num_workers = 3, local_devices = ('/job:worker/task:0',), communication = CollectiveCommunication.AUTO
INFO:tensorflow:Using MirroredStrategy with devices ('/job:worker/task:0',)
INFO:tensorflow:MultiWorkerMirroredStrategy with cluster_spec = {'worker': ['10.0.1.121:8890', '10.0.1.104:8890', '10.0.1.46:8890']}, task_type = 'worker', task_id = 0, num_workers = 3, local_devices = ('/job:worker/task:0',), communication = CollectiveCommunication.AUTO
Epoch 1/3
INFO:tensorflow:Collective batch_all_reduce: 7 all-red

In [3]:
# %tensorboard --logdir logs_rnn/fit --port 6006 --bind_all

In [4]:
# model.save('Minst_Lstm.model')